<a href="https://colab.research.google.com/github/Theieyrre/Hate-Speech-NLP/blob/main/%5BTRAIN_ONLY%5D_BERT_Sentiment_with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT Sentiment with Torch

### For jupyter install

In [ ]:
#%pip install torch
#%pip install pandas
#%pip install transformers
#%pip install numpy
#%pip install tqdm

### For Colab install:
Google Colab already has torch,numpy, tqdm and pandas installed. No need to install again

In [ ]:
%pip install transformers

     |████████████████████████████████| 778kB 3.4MB/s 
     |████████████████████████████████| 3.0MB 17.0MB/s 
     |████████████████████████████████| 1.1MB 27.0MB/s 
     |████████████████████████████████| 890kB 26.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b3888f19acacd8299cfd29095b8b76e40e0f70b2ded0b756fd9f48d073449569
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from collections import defaultdict

## Parameters

In [ ]:
# Data Parameters
h_na = '-'
h_filename = 'sentiment.csv'
h_c_sep = ','
h_index = 'id_str'
h_num_labels = 3

# BERT Parameters
h_preprocess_mode = 'dbmdz/bert-base-turkish-cased'
h_max_len = 280
h_batch_size = 16
h_epoch = 3 # Overfit after third epoch
h_text = 'tweet'
h_label = 'sentiment'

# Adam Optimizer Parameters
h_learning_rate = 2e-5
h_eps = 1e-8

## Import and prepare data
Import train file

In [ ]:
df = pd.read_csv(h_filename, na_values=h_na, sep=h_c_sep)
df = df.head(2000)
df.set_index(h_index, inplace=True)
df.head()

,date,tweet,screen_name,name,quote,label,sentiment,Unnamed: 8,Continue:,H2002
id_str,,,,,,,,,,
1.080414e+18,Jan 02 10 2019,@mithakanfidan Kızacaksınız ama söylemeliydim....,slmaleyku,Emine gngr AK 🇹🇷#sensörGelsinAcimDinsin,NoqayBalasi,NOT,neu,NaN,NaN,NaN
1.080401e+18,Jan 02 09 2019,"Müslüman, Müslümanın kardeşidir. ayetinin gölg...",gaziantepgm_,Gençlik Merkezi,NaN,NOT,pos,NaN,NaN,NaN
1.080401e+18,Jan 02 09 2019,@sadakte Bak sen söyledin yılbaşını bizim sarh...,spasaoglu,SALİH PAŞAOĞLU,NaN,NOT,neg,NaN,NaN,NaN
1.080473e+18,Jan 02 14 2019,Hasdane deyim adımının attığım her yer Suriyel...,AyseTek16,İnce Destek,NaN,NOT,neu,NaN,NaN,NaN
1.080473e+18,Jan 02 14 2019,Örnek. Kimyasal saldırıya karşı eylem yapan Su...,ozaneking,Ozan E. King,NaN,NOT,neu,NaN,NaN,NaN


In [ ]:
df[h_label].value_counts()

neg    1344
neu     489
pos     167
Name: sentiment, dtype: int64

## Label transform

In [ ]:
possible_labels = df[h_label].unique()
label_train_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_train_dict[possible_label] = index
df['category'] = df[h_label].replace(label_train_dict)
df.head()

,date,tweet,screen_name,name,quote,label,sentiment,Unnamed: 8,Continue:,H2002,category
id_str,,,,,,,,,,,
1.080414e+18,Jan 02 10 2019,@mithakanfidan Kızacaksınız ama söylemeliydim....,slmaleyku,Emine gngr AK 🇹🇷#sensörGelsinAcimDinsin,NoqayBalasi,NOT,neu,NaN,NaN,NaN,0
1.080401e+18,Jan 02 09 2019,"Müslüman, Müslümanın kardeşidir. ayetinin gölg...",gaziantepgm_,Gençlik Merkezi,NaN,NOT,pos,NaN,NaN,NaN,1
1.080401e+18,Jan 02 09 2019,@sadakte Bak sen söyledin yılbaşını bizim sarh...,spasaoglu,SALİH PAŞAOĞLU,NaN,NOT,neg,NaN,NaN,NaN,2
1.080473e+18,Jan 02 14 2019,Hasdane deyim adımının attığım her yer Suriyel...,AyseTek16,İnce Destek,NaN,NOT,neu,NaN,NaN,NaN,0
1.080473e+18,Jan 02 14 2019,Örnek. Kimyasal saldırıya karşı eylem yapan Su...,ozaneking,Ozan E. King,NaN,NOT,neu,NaN,NaN,NaN,0


## Loading Tokenize and Encoding

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
tokenizer = BertTokenizer.from_pretrained(h_preprocess_mode)

### Device control
Pick if CUDA available else use CPU, print for sanity check

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Data Loader

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

### Create Special DataSet

In [ ]:
class BERTDataset(Dataset):
  def __init__(self, text, label, tokenizer, max_len):
    self.text = text
    self.label = label
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.text)
  
  def __getitem__(self, item):
    text = str(self.text[item])
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        truncation=True,
        return_tensors='pt'
    )

    return {
        'text': text,
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labels': torch.tensor(self.label[item], dtype=torch.long)
    }

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = BERTDataset(
      text=df[h_text].to_numpy(),
      label=df['category'].to_numpy(),
      tokenizer=tokenizer,
      max_len=max_len
  )

  return DataLoader(
      ds,
      batch_size=batch_size,
      num_workers=4
  )

### Prepare train data

In [ ]:
dataloader_train = create_data_loader(df, tokenizer, h_max_len, h_batch_size)

## Build Classifier

In [ ]:
from transformers import BertForSequenceClassification
from transformers import BertModel

class Classifier(nn.Module):
  def __init__(self, n_classes):
    super(Classifier, self).__init__()
    self.bert = BertModel.from_pretrained(h_preprocess_mode)
    self.drop = nn.Dropout(0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    #output = self.out(output)
    #return self.softmax(output)
    return self.out(output)

In [ ]:
model = Classifier(h_num_labels)
model = model.to(device)

### Setting up optimizer

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
optimizer = AdamW(model.parameters(),
                  lr=h_learning_rate,
                  correct_bias=False, 
                  eps=h_eps)

#### Get Scheduler


In [ ]:
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader_train)*h_epoch)


#### Loss Function

In [ ]:
loss_fn = nn.CrossEntropyLoss().to(device)

## Training

In [ ]:
def train_epoch(
    model,
    dataloader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    n_examples
):
  model = model.train()
  losses = []
  correct_predictions = 0
  for d in tqdm(dataloader):
    input_ids = d['input_ids'].to(device)
    attention_mask = d['attention_mask'].to(device)
    labels = d['labels'].to(device)

    outputs = model(
        input_ids,
        attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, labels)

    correct_predictions += torch.sum(preds == labels)
    losses.append(loss)

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  
  return correct_predictions.double() / n_examples, torch.mean(torch.stack(losses))

# L O O P

In [ ]:
histroy = defaultdict(list)

epoch = 0
for epoch in tqdm(range(h_epoch), desc='Epoch {:1d}'.format(epoch)):
  train_acc, train_loss = train_epoch(
      model,
      dataloader_train,
      loss_fn,
      optimizer,
      device,
      scheduler,
      len(df)
  )

  tqdm.write(f'Train Loss: {train_loss}')
  tqdm.write(f'Train Acc: {train_acc}')

  histroy['train_acc'].append(train_acc)
  histroy['train_loss'].append(train_loss)

  torch.save(model.state_dict(), "model.bin")




Train Loss: 0.7547425627708435
Train Acc: 0.7135



Train Loss: 0.5842022895812988
Train Acc: 0.7915



Train Loss: 0.49800077080726624
Train Acc: 0.837


KeyboardInterrupt: ignored